
このノートブックを実行するには、次の追加ライブラリが必要です。 Colab での実行は実験的なものであることに注意してください。問題がある場合は、Github の問題を報告してください。


In [ ]:
!pip install d2l==1.0.0-beta0



# マルチヘッドアテンション

:label: `sec_multihead-attention`

実際には、同じクエリ、キー、および値のセットが与えられた場合、さまざまな範囲 (たとえば、短距離と長距離) の依存関係をキャプチャするなど、同じアテンション メカニズムのさまざまな動作からの知識をモデルに組み合わせることが必要になる場合があります。シーケンス内で。したがって、有益である可能性があります<br />これにより、アテンション メカニズムがクエリ、キー、値の異なる表現部分空間を共同で使用できるようになります。

この目的を達成するために、単一のアテンション プーリングを実行する代わりに、$h$ が独立して学習した線形射影を使用してクエリ、キー、および値を変換できます。次に、これらの $h$ 投影されたクエリ、キー、および値が並行してアテンション プーリングに供給されます。最終的に、$h$ アテンション プーリング出力が連結され、別の学習された線形射影と変換されて、最終出力が生成されます。この設計は*マルチヘッド アテンション*と呼ばれ、 $h$ アテンション プーリング出力のそれぞれが*ヘッド*です :cite: `Vaswani.Shazeer.Parmar.ea.2017` 。全結合層を使用して学習可能な線形変換を実行し、 :numref: `fig_multi-head-attention`はマルチヘッド アテンションを記述します。 

![](../img/multi-head-attention.svg) :label: `fig_multi-head-attention`


In [1]:
import math
import torch
from torch import nn
from d2l import torch as d2l


## モデル

マルチヘッド アテンションの実装を提供する前に、このモデルを数学的に形式化しましょう。クエリ $\mathbf{q} \in \mathbb{R}^{d_q}$、キー $\mathbf{k} \in \mathbb{R}^{d_k}$、値 $\mathbf{ v} \in \mathbb{R}^{d_v}$、各アテンションヘッド $\mathbf{h}_i$ ($i = 1, \ldots, h$) は次のように計算されます。

 $$\mathbf{h}_i = f(\mathbf W_i^{(q)}\mathbf q, \mathbf W_i^{(k)}\mathbf k,\mathbf W_i^{(v)}\mathbf v) \in \mathbb R^{p_v},$$

ここで、学習可能なパラメータ $\mathbf W_i^{(q)}\in\mathbb R^{p_q\times d_q}$、$\mathbf W_i^{(k)}\in\mathbb R^{p_k\times d_k}$ $\mathbf W_i^{(v)}\in\mathbb R^{p_v\times d_v}$、$f$ は、:numref: `sec_attention-scoring-functions`の加法的注意やスケーリングされたドット積注意などの注意プーリングです。 `sec_attention-scoring-functions` 。マルチヘッド アテンションの出力は、$h$ ヘッドの連結の学習可能なパラメーター $\mathbf W_o\in\mathbb R^{p_o\times hp_v}$ を介した別の線形変換です。

 $$\mathbf W_o \begin{bmatrix}\mathbf h_1\\vdots\\mathbf h_h\end{bmatrix} \in \mathbb{R}^{p_o}.$$

この設計に基づいて、各ヘッドは入力の異なる部分に対応できます。単純な加重平均よりも高度な関数を表現できます。

## 実装

私たちの実装では、マルチヘッド アテンションの [**各ヘッドに対してスケーリングされたドット積アテンションを選択**] します。計算コストとパラメータ化コストの大幅な増加を避けるために、$p_q = p_k = p_v = p_o / h$ を設定します。クエリ、キー、値の線形変換の出力数を $p_q h = p_k h = p_v h = p_o$ に設定すると、$h$ ヘッドを並列計算できることに注意してください。次の実装では、 $p_o$ は引数`num_hiddens`を介して指定されます。


In [2]:
class MultiHeadAttention(d2l.Module):  #@save
    """Multi-head attention."""
    def __init__(self, num_hiddens, num_heads, dropout, bias=False, **kwargs):
        super().__init__()
        self.num_heads = num_heads
        self.attention = d2l.DotProductAttention(dropout)
        self.W_q = nn.LazyLinear(num_hiddens, bias=bias)
        self.W_k = nn.LazyLinear(num_hiddens, bias=bias)
        self.W_v = nn.LazyLinear(num_hiddens, bias=bias)
        self.W_o = nn.LazyLinear(num_hiddens, bias=bias)

    def forward(self, queries, keys, values, valid_lens):
        # Shape of queries, keys, or values:
        # (batch_size, no. of queries or key-value pairs, num_hiddens)
        # Shape of valid_lens: (batch_size,) or (batch_size, no. of queries)
        # After transposing, shape of output queries, keys, or values:
        # (batch_size * num_heads, no. of queries or key-value pairs,
        # num_hiddens / num_heads)
        queries = self.transpose_qkv(self.W_q(queries))
        keys = self.transpose_qkv(self.W_k(keys))
        values = self.transpose_qkv(self.W_v(values))

        if valid_lens is not None:
            # On axis 0, copy the first item (scalar or vector) for num_heads
            # times, then copy the next item, and so on
            valid_lens = torch.repeat_interleave(
                valid_lens, repeats=self.num_heads, dim=0)

        # Shape of output: (batch_size * num_heads, no. of queries,
        # num_hiddens / num_heads)
        output = self.attention(queries, keys, values, valid_lens)
        # Shape of output_concat: (batch_size, no. of queries, num_hiddens)
        output_concat = self.transpose_output(output)
        return self.W_o(output_concat)


[**複数のヘッドの並列計算**] を可能にするために、上記の`MultiHeadAttention`クラスは、以下に定義されている 2 つの転置メソッドを使用します。具体的には、 `transpose_output`メソッドは、 `transpose_qkv`メソッドの動作を逆にします。


In [3]:
@d2l.add_to_class(MultiHeadAttention)  #@save
def transpose_qkv(self, X):
    """Transposition for parallel computation of multiple attention heads."""
    # Shape of input X: (batch_size, no. of queries or key-value pairs,
    # num_hiddens). Shape of output X: (batch_size, no. of queries or
    # key-value pairs, num_heads, num_hiddens / num_heads)
    X = X.reshape(X.shape[0], X.shape[1], self.num_heads, -1)
    # Shape of output X: (batch_size, num_heads, no. of queries or key-value
    # pairs, num_hiddens / num_heads)
    X = X.permute(0, 2, 1, 3)
    # Shape of output: (batch_size * num_heads, no. of queries or key-value
    # pairs, num_hiddens / num_heads)
    return X.reshape(-1, X.shape[2], X.shape[3])

@d2l.add_to_class(MultiHeadAttention)  #@save
def transpose_output(self, X):
    """Reverse the operation of transpose_qkv."""
    X = X.reshape(-1, self.num_heads, X.shape[1], X.shape[2])
    X = X.permute(0, 2, 1, 3)
    return X.reshape(X.shape[0], X.shape[1], -1)


キーと値が同じであるおもちゃの例を使用して、 `MultiHeadAttention`クラス**を [実装してテスト] してみ**ましょう。結果として、マルチヘッド アテンションの出力の形状は ( `batch_size` 、 `num_queries` 、 `num_hiddens` ) になります。


In [4]:
num_hiddens, num_heads = 100, 5
attention = MultiHeadAttention(num_hiddens, num_heads, 0.5)
batch_size, num_queries, num_kvpairs = 2, 4, 6
valid_lens = torch.tensor([3, 2])
X = torch.ones((batch_size, num_queries, num_hiddens))
Y = torch.ones((batch_size, num_kvpairs, num_hiddens))
d2l.check_shape(attention(X, Y, Y, valid_lens),
                (batch_size, num_queries, num_hiddens))


## まとめ

マルチヘッド アテンションは、クエリ、キー、値の異なる表現サブスペースを介して同じアテンション プーリングの知識を組み合わせます。マルチヘッド アテンションの複数のヘッドを並行して計算するには、適切なテンソル操作が必要です。

## 演習
1. この実験では、複数の頭部の注意の重みを視覚化します。
1. マルチヘッド アテンションに基づいてトレーニングされたモデルがあり、予測速度を向上させるために最も重要でないアテンション ヘッドを削除したいとします。注意頭の重要性を測定する実験をどのように計画できるでしょうか?



[ディスカッション](https://discuss.d2l.ai/t/1635)
